In [21]:
#import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import TextVectorization
import tensorflow_datasets as tfds


In [26]:
import os
import random
from pycocotools.coco import COCO
from PIL import Image

# === Paths ===
data_dir = 'data'
img_dir = os.path.join(data_dir, 'train2017','train2017')
ann_file = os.path.join(data_dir,'annotations_trainval2017', 'annotations', 'captions_train2017.json')

# === Load COCO captions ===
coco = COCO(ann_file)
img_ids = coco.getImgIds()

# === Choose 100 random image IDs ===
selected_img_ids = random.sample(img_ids, 100)

samples = []

for img_id in selected_img_ids:
    img_info = coco.loadImgs(img_id)[0]
    file_name = img_info['file_name']
    file_path = os.path.join(img_dir, file_name)

    # Load the first caption (or more if needed)
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    caption = anns[0]['caption'] if anns else ""

    # Load image using PIL (or use tf.io.read_file if you prefer tensors)
    image = Image.open(file_path).convert('RGB')

    samples.append({
        'image': image,
        'caption': caption,
        'file_path': file_path,
    })

# ✅ Now 'samples' is a list of 100 (image, caption) dictionaries


loading annotations into memory...
Done (t=1.50s)
creating index...
index created!


In [27]:
import tensorflow as tf

def load_image_tensor(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

# Convert to tf.data.Dataset if needed
file_paths = [sample['file_path'] for sample in samples]
captions = [sample['caption'] for sample in samples]

dataset = tf.data.Dataset.from_tensor_slices((file_paths, captions))

def process(path, caption):
    image = load_image_tensor(path)
    return {'image': image, 'captions': {'text': caption}}

dataset = dataset.map(process)


In [29]:
#image model (encoder)
encoder = EfficientNetB3(include_top=False,pooling='avg')
encoder.trainable = False
max_token = 10000
max_length = 30
vectorizer = TextVectorization(max_tokens=max_token, output_sequence_length=max_length+1)
captions = dataset.map(lambda x: x['captions']['text'])
vectorizer.adapt(captions)
vocab_size = vectorizer.vocabulary_size()

In [30]:
#feature pre processing

vocab_size

370

In [ ]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(2,), dtype=tf.string, name=None)>